# Librerías

In [1]:
#!pip install pysqlite3
#!pip install scikit-lear
!pip install numpy==1.23.5

In [2]:
# Manejo de Datos
import numpy as np
import pandas as pd
import sqlite3 as sql
import os
import sys
import datetime

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Estadísticas y Pruebas
import scipy.stats as stats
from scipy.stats import gaussian_kde

# Procesamiento de Datos
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Modelado y Algoritmos
from sklearn import neighbors
import joblib

# Interactividad
from ipywidgets import interact

# Google Colab
from google.colab import drive
from google.colab import files

# Otros
from collections import Counter


# Conectar con google drive


In [3]:
drive.flush_and_unmount() #Linea en caso de tener que desconectar el drive por algún tipo de falla

Drive not mounted, so nothing to flush and unmount.


In [4]:
#drive.flush_and_unmount()  #Linea en caso de tener que desconectar el drive por algún tipo de falla
drive.mount('/content/drive') #Linea para conectar al drive

Mounted at /content/drive


In [5]:
path="/content/drive/MyDrive/analitica 3/sistemas_recomendacion" ### ruta del repositorio en drive
os.chdir(path) ### volver la carpeta del repositorio directorio de trabajo
sys.path.append(path) ### agregarla al path, poder leer archivos de funciones propios como paquetes

In [6]:
import a_funciones as fn #Importar el documento de funciones para hacer uso de estas

# Base de datos

In [7]:
conn = sql.connect('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/data/db_movies3') #Crear la conexión con la base de datos
cur = conn.cursor() #Creacion del cursos para realizar consultas dentro del mismo SQL

In [8]:
# Creación de cursor para  ejecutar consultas en la base de datos
# Visualizar las tablas contenidas en la base de datos
cur.execute("SELECT name FROM sqlite_master where type='table'")
cur.fetchall()

[('df_terminado',),
 ('df_catalogo',),
 ('reco',),
 ('movies_final',),
 ('ratings_final',),
 ('df_explorado',)]

#Modelos

##Sistemas de recomendación basados en popularidad

In [9]:
#Peliculas mejor calificadas por más de 120 usuarios (TOP 50)
pd.read_sql("""select title, rating as rat,
                count (*) as count
                from df_terminado
                group by title
                having count > 120  -- Filtra solo las películas que tienen más de 120 calificaciones
                order by rat desc
                limit 50""",conn)   # Limita la salida a las 50 primeras películas (las mejores calificadas)

,title,rat,count
0,X-Men (2000),5.0,133
1,"Usual Suspects, The (1995)",5.0,204
2,Titanic (1997),5.0,140
3,"Terminator, The (1984)",5.0,131
4,Star Wars: Episode VI - Return of the Jedi (1983),5.0,196
5,Star Wars: Episode V - The Empire Strikes Back...,5.0,211
6,Star Wars: Episode IV - A New Hope (1977),5.0,251
7,Speed (1994),5.0,171
8,Seven (a.k.a. Se7en) (1995),5.0,203
9,Schindler's List (1993),5.0,220


In [10]:
#Peliculas con mejor promedio de calificación (TOP 20)
pd.read_sql("""select title, avg(rating) as avg_rat,
                count (*) as count
                from df_terminado
                group by title
                order by avg_rat desc
                limit 20""",conn)   # Limita la salida a las 20 películas con el promedio más alto

,title,avg_rat,count
0,Secrets & Lies (1996),4.590909,11
1,Guess Who's Coming to Dinner (1967),4.545455,11
2,Paths of Glory (1957),4.541667,12
3,"Streetcar Named Desire, A (1951)",4.475000,20
4,"Celebration, The (Festen) (1998)",4.458333,12
5,"Shawshank Redemption, The (1994)",4.445289,317
6,Ran (1985),4.433333,15
7,Yojimbo (1961),4.429102,13
8,His Girl Friday (1940),4.392857,14
9,Three Colors: Red (Trois couleurs: Rouge) (1994),4.379895,16


In [11]:
#Peliculas mejor calificadas en promedio por año de publicación
best_movyear=pd.read_sql("""select year_movies,title, avg(rating) as avg_rat,
                count(*) as count
                from df_explorado
                group by year_movies, title
                order by year_movies desc, avg_rat desc""",conn)

# Ahora con el DataFrame obtenido, seleccionamos la mejor película (mayor promedio de rating) aclarando que es una por cada año
best_movyear2= best_movyear.sort_values(['year_movies', 'avg_rat'], ascending=[False, False]) \
                      .groupby('year_movies', as_index=False).first()

best_movyear2

,year_movies,title,avg_rat,count
0,1940.0,His Girl Friday (1940),4.392857,28
1,1941.0,"Maltese Falcon, The (1941)",4.170455,88
2,1942.0,Casablanca (1942),4.240000,200
3,1944.0,Double Indemnity (1944),4.323529,51
4,1946.0,Notorious (1946),4.250000,60
...,...,...,...,...
72,2014.0,Nightcrawler (2014),4.166667,54
73,2015.0,Spotlight (2015),4.157895,19
74,2016.0,Arrival (2016),3.980769,26
75,2017.0,Logan (2017),4.280000,50


##Sistema de recomendación basado en contenido-Manual

In [12]:
# Carga completa de la tabla df_catalogo desde la base de dato
df_catalogo=pd.read_sql("""select * from df_catalogo""",conn)
df_catalogo

,movie_id,title,promedio_rating,year_movies,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),3.935248,0.705128,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,Grumpier Old Men (1995),3.467564,0.705128,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,6,Heat (1995),3.971356,0.705128,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,47,Seven (a.k.a. Se7en) (1995),4.031100,0.705128,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,50,"Usual Suspects, The (1995)",4.263023,0.705128,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100,85510,Sucker Punch (2011),3.125000,0.910256,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2101,93363,John Carter (2012),3.325302,0.923077,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2102,111364,Godzilla (2014),2.852179,0.948718,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2103,1327,"Amityville Horror, The (1979)",3.010443,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [13]:
# Preparación del dataframe para modelos de machine learning en este caso de recomendación
df_modelos=df_catalogo.drop(columns=['title','movie_id'])

# Guardamos el dataframe procesado 'df_modelos' en un archivo .joblib para usarlo después en posteriores modelos
joblib.dump(df_modelos,"/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_modelos.joblib")

['/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_modelos.joblib']

**Modelo con película en específico**

In [14]:
# Definimos la película de referencia para buscar películas similares
movie='Avengers: Infinity War - Part I (2018)'

ind_movie=df_catalogo[df_catalogo['title']==movie].index.values.astype(int)[0] ### indice de la película en dataframe escalado y dummificado
row_sel_movie =df_modelos.iloc[ind_movie,:] ### seleccionar fila de caracteristicas de la pelicula seleccionada
similar_movies=df_modelos.corrwith(row_sel_movie,axis=1) ### calcular correlación de catalogo vs película seleccionada
similar_movies=similar_movies.sort_values(ascending=False) ### ordener películas de mayor a menor correlación
top_similar_movies = similar_movies.to_frame(name="correlación").iloc[0:10,] #### Convertimos a DataFrame y agregamos los títulos
top_similar_movies['title'] = df_catalogo['title']

In [15]:
# Mostrar películas mas correlacionadas a nuestro ejemplo específico
top_similar_movies

,correlación,title
1469,1.000000,Avengers: Infinity War - Part I (2018)
1043,0.999990,Guardians of the Galaxy 2 (2017)
1036,0.999861,X-Men: Days of Future Past (2014)
1470,0.999858,Thor: Ragnarok (2017)
820,0.999835,Guardians of the Galaxy (2014)
1042,0.999619,Ant-Man (2015)
797,0.999608,Iron Man (2008)
2092,0.999592,Black Panther (2017)
824,0.999543,Doctor Strange (2016)
1041,0.999284,Avengers: Age of Ultron (2015)


**Modelo general para cualquier película**

In [ ]:
# Modelo general como funcion que toma como entrada el título de las películas
def sist_recomendacion(movie=list(df_catalogo['title'])):
    ind_movie=df_catalogo[df_catalogo['title']==movie].index.values.astype(int)[0] ### indice de la película en dataframe escalado y dummificado
    similar_movies=df_modelos.corrwith(df_modelos.iloc[ind_movie,:],axis=1) ##calcular correlación de catalogo vs película seleccionada
    similar_movies=similar_movies.sort_values(ascending=False) ###  ordener películas de mayor a menor correlación
    top_similar_movies=similar_movies.to_frame(name="correlación").iloc[0:10,] ### Seleccionamos las 10 mejores películas recomendadas
    top_similar_movies['title']=df_catalogo["title"] ### agregar los nombres de las peliculas

    return top_similar_movies

print(interact(sist_recomendacion)) # Activamos el selector interactivo para elegir una película y obtener recomendaciones


## Sistema de recomendación basado en contenido KNN

In [17]:
#Se reinicia el índice de ambos DataFrames para que esten alineados y no den problemas
df_modelos = df_modelos.reset_index(drop=True)
df_catalogo = df_catalogo.reset_index(drop=True)

In [18]:
#Se crea el modelo KNN con 10 vecinos y usando la distancia euclidiana
model=neighbors.NearestNeighbors(n_neighbors=11,metric='euclidean')
model.fit(df_modelos)
dist,idlist=model.kneighbors(df_modelos) # se calculan las distancias y los indices de los 10 vecinos más cercano

In [19]:
#Convertimos las matrices dist (distancias) e idlist (índices de vecinos) en dataframes de pandas para facilitar su uso posterior
distancias=pd.DataFrame(dist)
id_list=pd.DataFrame(idlist)

**Modelo con película en específico**

In [20]:
movie_list_name = []  # Lista vacía para almacenar títulos recomendados
movie_name='Avengers: Infinity War - Part I (2018)' # Película ejemplo
movie_id = df_catalogo[df_catalogo['title'] == movie_name].index ### extraer el indice de la película
movie_id = movie_id[0] ## si encuentra varios solo guarde uno


# Guardar la lista de peliculas recomendadas
for newid in idlist[movie_id]:
    movie_list_name.append(df_catalogo.loc[newid].title) ### agrega el nombre de cada una de los id recomendados

# Mostrar películas mas parecidas nuestro ejemplo específico
movie_list_name


['Avengers: Infinity War - Part I (2018)',
 'Guardians of the Galaxy 2 (2017)',
 'Guardians of the Galaxy (2014)',
 'Thor: Ragnarok (2017)',
 'Serenity (2005)',
 'X-Men: Days of Future Past (2014)',
 'Iron Man (2008)',
 'Black Panther (2017)',
 'Ant-Man (2015)',
 'Doctor Strange (2016)',
 'X-Men (2000)']

**Modelo general para cualquier película**

In [ ]:
# Modelo general como función que toma como entrada el título de las películas
def movie_recomender(movie_name=list(df_catalogo['title'].value_counts().index)):
    movie_list_name = []
    movie_id = df_catalogo[df_catalogo['title'] == movie_name].index ### extraer el indice del libro
    movie_id=movie_id[0] ## si encuentra varios solo guarde uno

# Iterar sobre los 10 vecinos de esa película para evitar titulos duplicados
    for newid in idlist[movie_id]:
        movie_list_name.append(df_catalogo.loc[newid].title) ### agrega el nombre de cada una de los id recomendados
    return movie_list_name

print(interact(movie_recomender)) # Activamos el selector interactivo para elegir una película y obtener recomendaciones